# Single source shortest paths

* Weighted graph:
  - $G = (V, E)$
  - $W : E \rightarrow R$
* Single source shortest path
  - Find shortest paths from a fixed vertex to every other vertex
* Assume, for now, that edge weights are all non-negative

![Graph](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/wg-2.png?alt=media&token=f5168a8e-5654-4f75-be8f-5396fbf43abe)

-------------------------------------------------------------------------------
*  Compute shortest paths from $0$ to all other vertices
* Imagine vertices are oil depots, edges are pipelines
* Set fire to oil depot at vertex $0$
* Fire travels at uniform speed along each pipeline
* First oil depot to catch fire after $0$ is nearest vertex
* Next oil depot is second nearest vertex
* $...$
-------------------------------------------------------------------------------
* Compute expected burn time for each vertex
* Each time a new vertex burns, update the expected burn times of its neighbours
* Algorithm due to [Edsger W Dikjstra](https://en.wikipedia.org/wiki/Edsger_W._Dijkstra)


### Dijkstra's algorithm: Proof of correctness

* It is a greedy strategy
  - So this kind of a short sighted strategy where we are ignoring anything else
  - We never go back and make a choice that we had to take a choice we had taken before and undo it
  - So, this kind of forward thinking strategy where we keep making a local decision and never go back on it is called greedy strategy (or greedy approach)
* Each new shortest path we discover extends an earlier one
* By induction, assume we have found shortest paths to all the vertices already burnt
* Next vertex to burn is **v**, via **x**
* Cannot find a shorter path later from **y** to **v** via **w**
  - Burn time of **w** $\geq$ burn time of **v**
  - Edge from **w** to **v** has weight $\geq$ 0
* This argument breaks down if edge (**w**, **v**) can have negative weight
  - Can't use Dijkstra's algorithm with negative edge weights

![Graph](https://firebasestorage.googleapis.com/v0/b/fb-sandbox-25.appspot.com/o/wg-5.png?alt=media&token=539e76a2-746a-4ac7-8c3a-e8be99a2f168)

### Implementation

* Maintain two dictionaries with vertices as they keys
  - `visited`, intially $False$ for all `v` (burnt vertices)
  - `distance`, initially $infinity$ for all `v` (expected burn time)
* Set `distance[s]` to $0$
* Repeat, until all reachable vertices are visited
  - Find unvisited vertex `nextv` with minimum distance
  - Set `visited[nextv]` to $True$
  - Recompute `distance[v]` for every neighbour `v` of `nextv`

### Complexity

* Setting $infinity$ takes `O(n^2)` time
* Main loop runs `n` times
  - Each iteration visits one more vertex
  - $O(n)$ to find the next vertex to visit
  - $O(n)$ to update `distance[v]` for neighbours
* Overall $O(n^2)$
* If we use an adjacency list
  - Setting $infinity$ and updating distance both $O(m)$, amortised
  - $O(n)$ bottleneck remains to find next vertex to visit
  - A better data structure is coming up $...$

In [ ]:
# Adjacency matrix implementation
def dijkstra(WMat, s):
  (rows, cols, x) = WMat.shape
  infinity = np.max(WMat) * rows + 1
  (visited, distance) = ({}, {})

  for v in range(rows):
    (visited[v], distance[v]) = (False, infinity)
  
  distance[s] = 0

  for u in range(rows):
    nextd = min([distance[v] for v in range(rows)
                      if not visited[v]])
    nextvlist = [v for v in range(rows)
                    if (not visited[v]) and
                        distance[v] == nextd]
    
    if nextvlist == []:
      break
    
    nextv = min(nextvlist)
    visited[nextv] = True

    for v in range(cols):
      if WMat[nextv, v, 0] == 1 and (not visited[v]):
        distance[v] = min(distance[v], distance[nextv] + WMat[nextv, v, 1])
  
  return distance

In [ ]:
# Adjacency list implementation
def dijkstra(WList, s):
  infinity = 1 + len(WList.keys()) * max([d for u in WList.keys()
                                              for (v, d) in WList[u]])
  (visited, distance) = ({}, {})

  for v in WList.keys():
    (visited[v], distance[v]) = (False, infinity)
  
  distance[s] = 0

  for u in WList.keys():
    nextd = min([distance[v] for v in WList.keys()
                      if not visited[v]])
    nextvlist = [v for v in WList.keys()
                    if (not visited[v]) and
                        distance[v] == nextd]
    if nextvlist == []:
      break
    
    nextv = min(nextvlist)
    visited[nextv] = True

    for (v, d) in WList[nextv]:
      if not visited[v]:
        distance[v] = min(distance[v], distance[nextv] + d)
  
  return distance

### Summary

* Dijkstra's algorithm computes single source shortest paths
* Use a greedy strategy to identify vertices to visit
  - Next vertex to visit is based on shortest distance computed so far
  - Need to prove that such a strategy is correct
  - Correctness requires edge weights to be non-negative
* Complexity is $O(n^2)$
  - Even with adjacency lists
  - Bottleneck is identifying unvisited vertex with minimum distance
  - Need a better data structure to identify and remove minimum (or maximum) from a collection